## Hot Music Recommender

1. Check whether or not the song is in the Billboard Hot 100. 
    - If the song is in the Billboard Hot 100, recommend another song from there. 
    - If the song is not in the Billboard Hot 100, skip to step 2.


2. Collect the audio features from that song by sending a requesto to the Spotify API.
    - "Predict" the cluster of the song.
    - Pick a random song from the predicted cluster and give it back to the user.

### Importing all libraries & spotify authentification

In [67]:
import pandas as pd
import numpy as np
import json
import pprint
import math

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import spotipy 
from client_ID_spotify import *
from spotipy.client import Spotify

from spotipy.oauth2 import SpotifyClientCredentials

from client_ID_spotify import Client_ID, Client_Secret

In [68]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id = Client_ID,
                                                           client_secret = Client_Secret))

### Loading needed dataframes

In [14]:
%store -r audio_feat_all

In [6]:
%store -r spotify_songs

In [12]:
%store -r audiofeat_hot100

### Importing the model (scaler and kmeans using Ignacio function)

In [64]:
import pickle

def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
    except FileNotFoundError: 
        print("File not found!") 

In [87]:
scaler = load(filename = "scaler.pickle")

In [88]:
kmeans = load(filename = "kmeans_4.pickle")

In [15]:
audio_feat_all

,artist,song,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,origin,cluster
0,PNAU,Wild Strawberries,6pGUGTIaZ1H4jKHIL4Fged,0.647,0.933,7,-4.056,1,0.1110,0.000351,0.002770,0.3340,0.332,119.921,235107,4,0.0,4
1,Stromae,Papaoutai,09TcIuH1ZO7i4vicWKoaN2,0.733,0.818,10,-7.222,0,0.0859,0.024100,0.000000,0.0636,0.253,116.019,232147,4,0.0,6
2,Eurythmics,Sweet Dreams (Are Made of This) - Remastered,1TfqLAPs4K3s2rJMoCokcS,0.692,0.711,0,-7.498,0,0.0317,0.225000,0.000000,0.1200,0.875,125.135,216933,4,0.0,6
3,Led Zeppelin,Rock and Roll - 1990 Remaster,3w2GGz0HjIu9OcWXINRFJR,0.327,0.895,9,-7.428,1,0.0367,0.000564,0.015900,0.1040,0.898,169.390,219800,4,0.0,4
4,Jason Derulo,Talk Dirty (feat. 2 Chainz),4X4tgBEUiT6WqB2oTJ5ynH,0.760,0.652,6,-7.321,1,0.2320,0.034800,0.000000,0.3070,0.759,100.315,177685,4,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Lainey Wilson,Things A Man Oughta Know,5QS8PNEWbqTEZyQ6e9ZbJf,0.659,0.683,3,-5.623,1,0.0312,0.513000,0.000005,0.1330,0.397,139.931,203373,4,1.0,0
96,Thomas Rhett,Country Again,3CiF9D8iuVhZWDvvjf8HpG,0.467,0.481,2,-6.947,1,0.0255,0.227000,0.000189,0.1100,0.248,147.886,221240,4,1.0,4
97,Elle King & Miranda Lambert,Drunk (And I Don't Wanna Go Home),0QULNNd9z5s35entfiiXoa,0.612,0.884,5,-4.400,1,0.0459,0.005160,0.000000,0.0997,0.626,119.991,245626,4,1.0,0
98,Lil Baby & Lil Durk,If You Want To,1EXRmdtTA2IEnYGiuw9plD,0.795,0.599,6,-7.819,1,0.1230,0.041400,0.000000,0.1930,0.351,97.033,195454,4,1.0,0


In [13]:
audiofeat_hot100

,artist,song,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,BTS,Butter,0.759,0.459,8,-5.187,1,0.0948,0.00323,0.000000,0.0906,0.695,109.997,audio_features,2bgTY4UwhfBYhGT4HUYStN,spotify:track:2bgTY4UwhfBYhGT4HUYStN,https://api.spotify.com/v1/tracks/2bgTY4UwhfBY...,https://api.spotify.com/v1/audio-analysis/2bgT...,164442,4
1,Olivia Rodrigo,Good 4 U,0.556,0.661,6,-5.052,0,0.2040,0.30000,0.000000,0.1010,0.668,168.560,audio_features,6PERP62TejQjgHu81OHxgM,spotify:track:6PERP62TejQjgHu81OHxgM,https://api.spotify.com/v1/tracks/6PERP62TejQj...,https://api.spotify.com/v1/audio-analysis/6PER...,178148,4
2,Dua Lipa Featuring DaBaby,Levitating,0.702,0.825,6,-3.787,0,0.0601,0.00883,0.000000,0.0674,0.915,102.977,audio_features,463CkQjx2Zk1yXoBuierM9,spotify:track:463CkQjx2Zk1yXoBuierM9,https://api.spotify.com/v1/tracks/463CkQjx2Zk1...,https://api.spotify.com/v1/audio-analysis/463C...,203064,4
3,Justin Bieber Featuring Daniel Caesar & Giveon,Peaches,0.677,0.696,0,-6.181,1,0.1190,0.32100,0.000000,0.4200,0.464,90.030,audio_features,4iJyoBOLtHqaGxP12qzhQI,spotify:track:4iJyoBOLtHqaGxP12qzhQI,https://api.spotify.com/v1/tracks/4iJyoBOLtHqa...,https://api.spotify.com/v1/audio-analysis/4iJy...,198082,4
4,Silk Sonic (Bruno Mars & Anderson .Paak),Leave The Door Open,0.586,0.616,5,-7.964,1,0.0324,0.18200,0.000000,0.0927,0.719,148.088,audio_features,7MAibcTli4IisCtbHKrGMh,spotify:track:7MAibcTli4IisCtbHKrGMh,https://api.spotify.com/v1/tracks/7MAibcTli4Ii...,https://api.spotify.com/v1/audio-analysis/7MAi...,242096,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Lainey Wilson,Things A Man Oughta Know,0.659,0.683,3,-5.623,1,0.0312,0.51300,0.000005,0.1330,0.397,139.931,audio_features,5QS8PNEWbqTEZyQ6e9ZbJf,spotify:track:5QS8PNEWbqTEZyQ6e9ZbJf,https://api.spotify.com/v1/tracks/5QS8PNEWbqTE...,https://api.spotify.com/v1/audio-analysis/5QS8...,203373,4
96,Thomas Rhett,Country Again,0.467,0.481,2,-6.947,1,0.0255,0.22700,0.000189,0.1100,0.248,147.886,audio_features,3CiF9D8iuVhZWDvvjf8HpG,spotify:track:3CiF9D8iuVhZWDvvjf8HpG,https://api.spotify.com/v1/tracks/3CiF9D8iuVhZ...,https://api.spotify.com/v1/audio-analysis/3CiF...,221240,4
97,Elle King & Miranda Lambert,Drunk (And I Don't Wanna Go Home),0.612,0.884,5,-4.400,1,0.0459,0.00516,0.000000,0.0997,0.626,119.991,audio_features,0QULNNd9z5s35entfiiXoa,spotify:track:0QULNNd9z5s35entfiiXoa,https://api.spotify.com/v1/tracks/0QULNNd9z5s3...,https://api.spotify.com/v1/audio-analysis/0QUL...,245626,4
98,Lil Baby & Lil Durk,If You Want To,0.795,0.599,6,-7.819,1,0.1230,0.04140,0.000000,0.1930,0.351,97.033,audio_features,1EXRmdtTA2IEnYGiuw9plD,spotify:track:1EXRmdtTA2IEnYGiuw9plD,https://api.spotify.com/v1/tracks/1EXRmdtTA2IE...,https://api.spotify.com/v1/audio-analysis/1EXR...,195454,4


In [9]:
spotify_songs.head()

,artist,song,album,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,origin
0,PNAU,Wild Strawberries,PNAU (Tour Edition),6pGUGTIaZ1H4jKHIL4Fged,0.647,0.933,7,-4.056,1,0.1110,0.000351,0.00277,0.3340,0.332,119.921,235107,4,0
1,Stromae,Papaoutai,racine carrée,09TcIuH1ZO7i4vicWKoaN2,0.733,0.818,10,-7.222,0,0.0859,0.024100,0.00000,0.0636,0.253,116.019,232147,4,0
2,Eurythmics,Sweet Dreams (Are Made of This) - Remastered,Sweet Dreams (Are Made Of This),1TfqLAPs4K3s2rJMoCokcS,0.692,0.711,0,-7.498,0,0.0317,0.225000,0.00000,0.1200,0.875,125.135,216933,4,0
3,Led Zeppelin,Rock and Roll - 1990 Remaster,Led Zeppelin IV,3w2GGz0HjIu9OcWXINRFJR,0.327,0.895,9,-7.428,1,0.0367,0.000564,0.01590,0.1040,0.898,169.390,219800,4,0
4,Jason Derulo,Talk Dirty (feat. 2 Chainz),Tattoos (Special Edition),4X4tgBEUiT6WqB2oTJ5ynH,0.760,0.652,6,-7.321,1,0.2320,0.034800,0.00000,0.3070,0.759,100.315,177685,4,0


In [17]:
audio_feat_all[audio_feat_all["origin"] == 1] #hot 100

,artist,song,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,origin,cluster
0,BTS,Butter,2bgTY4UwhfBYhGT4HUYStN,0.759,0.459,8,-5.187,1,0.0948,0.00323,0.000000,0.0906,0.695,109.997,164442,4,1.0,0
1,Olivia Rodrigo,Good 4 U,6PERP62TejQjgHu81OHxgM,0.556,0.661,6,-5.052,0,0.2040,0.30000,0.000000,0.1010,0.668,168.560,178148,4,1.0,6
2,Dua Lipa Featuring DaBaby,Levitating,463CkQjx2Zk1yXoBuierM9,0.702,0.825,6,-3.787,0,0.0601,0.00883,0.000000,0.0674,0.915,102.977,203064,4,1.0,6
3,Justin Bieber Featuring Daniel Caesar & Giveon,Peaches,4iJyoBOLtHqaGxP12qzhQI,0.677,0.696,0,-6.181,1,0.1190,0.32100,0.000000,0.4200,0.464,90.030,198082,4,1.0,0
4,Silk Sonic (Bruno Mars & Anderson .Paak),Leave The Door Open,7MAibcTli4IisCtbHKrGMh,0.586,0.616,5,-7.964,1,0.0324,0.18200,0.000000,0.0927,0.719,148.088,242096,4,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Lainey Wilson,Things A Man Oughta Know,5QS8PNEWbqTEZyQ6e9ZbJf,0.659,0.683,3,-5.623,1,0.0312,0.51300,0.000005,0.1330,0.397,139.931,203373,4,1.0,0
96,Thomas Rhett,Country Again,3CiF9D8iuVhZWDvvjf8HpG,0.467,0.481,2,-6.947,1,0.0255,0.22700,0.000189,0.1100,0.248,147.886,221240,4,1.0,4
97,Elle King & Miranda Lambert,Drunk (And I Don't Wanna Go Home),0QULNNd9z5s35entfiiXoa,0.612,0.884,5,-4.400,1,0.0459,0.00516,0.000000,0.0997,0.626,119.991,245626,4,1.0,0
98,Lil Baby & Lil Durk,If You Want To,1EXRmdtTA2IEnYGiuw9plD,0.795,0.599,6,-7.819,1,0.1230,0.04140,0.000000,0.1930,0.351,97.033,195454,4,1.0,0


In [18]:
audio_feat_all[audio_feat_all["origin"] == 0] #spotify

,artist,song,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,origin,cluster
0,PNAU,Wild Strawberries,6pGUGTIaZ1H4jKHIL4Fged,0.647,0.933,7,-4.056,1,0.1110,0.000351,0.002770,0.3340,0.332,119.921,235107,4,0.0,4
1,Stromae,Papaoutai,09TcIuH1ZO7i4vicWKoaN2,0.733,0.818,10,-7.222,0,0.0859,0.024100,0.000000,0.0636,0.253,116.019,232147,4,0.0,6
2,Eurythmics,Sweet Dreams (Are Made of This) - Remastered,1TfqLAPs4K3s2rJMoCokcS,0.692,0.711,0,-7.498,0,0.0317,0.225000,0.000000,0.1200,0.875,125.135,216933,4,0.0,6
3,Led Zeppelin,Rock and Roll - 1990 Remaster,3w2GGz0HjIu9OcWXINRFJR,0.327,0.895,9,-7.428,1,0.0367,0.000564,0.015900,0.1040,0.898,169.390,219800,4,0.0,4
4,Jason Derulo,Talk Dirty (feat. 2 Chainz),4X4tgBEUiT6WqB2oTJ5ynH,0.760,0.652,6,-7.321,1,0.2320,0.034800,0.000000,0.3070,0.759,100.315,177685,4,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Lana Del Rey,Yayo,6EfNun62AXVq3on5hqKEWu,0.506,0.201,3,-12.931,0,0.0294,0.926000,0.000411,0.2530,0.145,71.925,321054,4,0.0,2
9996,Lana Del Rey,Yayo,2rldjI2fK6nYwlMgaITluf,0.506,0.201,3,-12.931,0,0.0294,0.926000,0.000411,0.2530,0.145,71.925,321054,4,0.0,2
9997,Nicki Minaj,Regret In Your Tears,0j8zLbIueSkVqB3Tl6PsjA,0.770,0.477,0,-10.972,0,0.0802,0.749000,0.208000,0.1660,0.248,106.128,229573,4,0.0,2
9998,Riot Shift,OBLIVION - Anderex Remix,6T4jfMT7vzbpjm8durkojq,0.329,0.977,8,-4.688,1,0.4230,0.001200,0.008310,0.2980,0.286,161.017,212633,4,0.0,5


## Music Recommender Function

In [ ]:
#Get user input
song_input = input("Enter the song title: ")
artist_input = input("Enter the song artist: ")

In [99]:
def hot_or_not():
    song = input("Enter the song title: ")
    
    if song.lower() in audio_feat_all[audio_feat_all["origin"] == 1]['song'].str.lower().to_list():

        user_cluster = int(audio_feat_all[audio_feat_all["song"].str.lower() == song.lower()]['cluster']) 

        user_recommendation = audio_feat_all[(audio_feat_all["origin"] == 1) & (audio_feat_all["cluster"] == user_cluster)].sample()

        print("The song you entered is in the hottest 100. Ckeck out this song: \n"+ list(user_recommendation["song"])[0]+" by", list(user_recommendation["artist"])[0],".")


    else:
    #Input is not in the hottest 100, then ask for artist to check if song+artist 

        artist = input("Who is the artist of the song? ")

        results = sp.search(q="track:"+song+" artist:"+artist,limit=5)
        uri = results['tracks']['items'][0]["uri"]

        result_audio_feat = sp.audio_features(uri)

        result_audio_feat_df = pd.DataFrame(result_audio_feat)
        result_audio_feat_df = result_audio_feat_df[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
           'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']]

        result_scaled = scaler.transform(result_audio_feat_df) 

        recommendation = kmeans.predict(result_scaled)

        user_recommendation = audio_feat_all[(audio_feat_all["origin"] == 0) & (audio_feat_all["cluster"] == recommendation[0])].sample()

        print("The song you entered is not in the hottest 100. Ckeck out this song: \n"+ list(user_recommendation["song"])[0]+" by", list(user_recommendation["artist"])[0],".")



In [101]:
hot_or_not()

Enter the song title: buTTer
The song you entered is in the hottest 100. Ckeck out this song: 
Build A Bitch by Bella Poarch .


In [95]:
result_audio_feat_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.338,0.34,9,-12.049,0,0.0339,0.58,0.0032,0.116,0.197,82.433,482830


In [49]:
# WE NEED: song name, artist name and URI
results

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3AStairway+to+Heaven+artist%3Aled&type=track&offset=0&limit=5',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/36QJpDe2go2KgaRleHCDTp'},
       'href': 'https://api.spotify.com/v1/artists/36QJpDe2go2KgaRleHCDTp',
       'id': '36QJpDe2go2KgaRleHCDTp',
       'name': 'Led Zeppelin',
       'type': 'artist',
       'uri': 'spotify:artist:36QJpDe2go2KgaRleHCDTp'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',


In [28]:
#Check if the song is part of the hot 100
def hot_or_not(song):
    
    
    if song in audio_feat_all[audio_feat_all["origin"] == 1]['song'].str.lower().to_list():
        
 
        print(f"The song you entered is in the hottest 100. Ckeck out this song: \n{rec_song} by {rec_artist}.")
          
            
        else:
            pass

SyntaxError: invalid syntax (<ipython-input-28-e81edc3b5dde>, line 11)